#### Matrix Factorization(MF)

---------------------

In [20]:
import numpy as np
import pandas as pd


# 데이터 불러오기
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('u.user', sep='|', names=u_cols, encoding='latin-1')
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
movies = pd.read_csv('u.item', sep='|', names=i_cols, encoding='latin-1')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('u.data', sep='\t', names=r_cols, encoding='latin-1')

# timestamp 제거 
ratings = ratings.drop('timestamp', axis=1)
movies = movies[['movie_id', 'title']]

In [21]:
# MF class

class MF():
    def __init__(self, ratings, K, alpha, beta, iterations, verbose=True):
        self.R = np.array(ratings)
        self.num_users, self.num_items = np.shape(self.R)
        self.K = K # 잠재요인(latent factor) 수
        self.alpha = alpha # 학습률
        self.beta = beta # 정규화 계수
        self.iterations = iterations #S zGD 계산 시 반복 횟수
        self.verbose = verbose # SGD 중간 학습과정 출력
    

    # RMSE
    def rmse(self):
        xs, ys = self.R.nonzero() # 평점이 있는 요소의 인덱스 가져오기
        self.predictions = []
        self.errors = []
        
        for x, y in zip(xs, ys):
            prediction = self.get_prediction(x, y)
            self.predictions.append(prediction)
            self.errors.append(self.R[x, y] - prediction) # 실제값-예측값 리스트에 추가
        self.predictions = np.array(self.predictions)
        self.errors = np.array(self.errors)
        return np.sqrt(np.mean(self.errors**2))

    def train(self): 

        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K)) 
        # 행렬을 평균 0, 표준편차 1/K인 정규분포를 가지는 난수로 초기화
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # users/items 평가경향을 0으로 초기화
        self.b_u = np.zeros(self.num_users)
        self.b_d = np.zeros(self.num_items)
        self.b = np.mean(self.R[self.R.nonzero()])

        # 평점이 있는 요소 인덱스 가져오기
        rows, columns = self.R.nonzero()
        # SGD 적용대상(평점이 있는 요소 인덱스, 평점)을 리스트로 저장
        self.samples = [(i, j, self.R[i,j]) for i, j in zip(rows, columns)]

        # SGD를 한 번 실행할 때마다 rmse가 얼마나 개선되는지 기록
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            rmse = self.rmse()
            training_process.append((i+1, rmse))
            if self.verbose:
                if (i+1) % 10 == 0:
                    print("Iteration: %d ; Train RMSE = %.4f " % (i+1, rmse))
        return training_process

    # 평점 예측값 구하는 함수
    def get_prediction(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i, :].dot(self.Q[j, :].T)
        # 사용자i, 아이템j에 대한 평점 예측치
        return prediction

    # SGD(Stochastic gradient descent) 실행
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_prediction(i, j) # 사용자i, 아이템j에 대한 평점 예측치
            e = (r - prediction) # 오차

            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i]) # 사용자 평가경향 업데이트
            self.b_d[j] += self.alpha * (e - self.beta * self.b_d[j]) # 아이템 평가경향 업데이트

            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:]) # P행렬 업데이트
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:]) # Q행렬 업데이트


In [22]:
# 전체 데이터 사용 MF

R_temp = ratings.pivot(index = 'user_id', columns = 'movie_id', values = 'rating').fillna(0)
mf = MF(R_temp, K = 30, alpha = 0.001, beta = 0.02, iterations = 100, verbose = True)
train_process = mf.train()

Iteration: 10 ; Train RMSE = 0.9585 
Iteration: 20 ; Train RMSE = 0.9373 
Iteration: 30 ; Train RMSE = 0.9280 
Iteration: 40 ; Train RMSE = 0.9224 
Iteration: 50 ; Train RMSE = 0.9183 
Iteration: 60 ; Train RMSE = 0.9143 
Iteration: 70 ; Train RMSE = 0.9096 
Iteration: 80 ; Train RMSE = 0.9031 
Iteration: 90 ; Train RMSE = 0.8941 
Iteration: 100 ; Train RMSE = 0.8821 


--------------------------

In [ ]:
#### train/test set 분리